## Probability that one set of dice rolls is greater than or equal to a given set

[StackExchange question.](https://math.stackexchange.com/questions/4379856/probability-that-one-set-of-dice-rolls-is-greater-than-or-equal-to-a-given-set)

### Single ability

First, we generate the distribution of a single ability score of 4d6, keep 3 highest. This is small enough for brute force enumeration of all possibilities, though more efficient algorithms do exist. More on this later.

In [1]:
import piplite
await piplite.install("hdroller")

import hdroller

single_ability = hdroller.d6.keep_highest(4, 3)

print(single_ability)

Denominator: 1296
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       3 |      1 |   0.077160% |
|       4 |      4 |   0.308642% |
|       5 |     10 |   0.771605% |
|       6 |     21 |   1.620370% |
|       7 |     38 |   2.932099% |
|       8 |     62 |   4.783951% |
|       9 |     91 |   7.021605% |
|      10 |    122 |   9.413580% |
|      11 |    148 |  11.419753% |
|      12 |    167 |  12.885802% |
|      13 |    172 |  13.271605% |
|      14 |    160 |  12.345679% |
|      15 |    131 |  10.108025% |
|      16 |     94 |   7.253086% |
|      17 |     54 |   4.166667% |
|      18 |     21 |   1.620370% |



### Problem 1

What's the chance that the sum of player A's six ability scores is greater than or equal to the sum of player B's six ability scores?
Addition of probability distributions can be done via convolution, so this is pretty simple.

In [2]:
print(6 @ single_ability >= 6 @ single_ability)

Denominator: 22452257707354557240087211123792674816
| Outcome |                                 Weight | Probability |
|--------:|---------------------------------------:|------------:|
|   False | 10773601417436608285167797336637018642 |  47.984490% |
|    True | 11678656289917948954919413787155656174 |  52.015510% |



### Problem 2

Now for the harder problem: what is the chance that there is some pairing of player A's ability scores and player B's ability scores
such that player A's score is greater than or equal to player B's score for each pair? The trick is to express the problem not over the six pairs, but the values the scores can take.
To wit: what is the chance, that for all values from 18 to 3, player A will have at least as many scores of at least that value as player B? (The equivalency is left as an exercise for the reader.)

From here, it turns out we can efficiently solve dice pool problems as long as we phrase the evaluation as
a series of iterative state transitions over tuples of (outcome, how many dice in each pool rolled that outcome)
and keep the number of states to a minimum. In this case the "dice" in the pool are entire ability scores rather than individual d6s.
[Here's an (unfortuantely still crude) explanation of the underlying algorithm.](https://gitlab.com/highdiceroller/hdroller/-/wikis/dice%20pool%20algorithm)

In [3]:
class SortedAllGe(hdroller.EvalPool):
    def next_state(self, state, outcome, a, b):
        # state is how many dice A has of outcome or higher minus how many dice B has of outcome or higher,
        # but "sticks" at -1 if it ever goes negative, indicating that B had a higher paired die at some point.
        if state is None:
            state = 0
        elif state < 0:
            return -1
        state += a - b
        if state < 0:
            return -1
        else:
            return state
        
    def final_outcome(self, final_state, *_):
        return final_state >= 0
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1
    
evaluator = SortedAllGe()
print(evaluator.eval(single_ability.pool(6), single_ability.pool(6)))

Denominator: 22452257707354557240087211123792674816
| Outcome |                                 Weight | Probability |
|--------:|---------------------------------------:|------------:|
|   False | 17750571119832007830956108494444318705 |  79.059181% |
|    True |  4701686587522549409131102629348356111 |  20.940819% |



Checking this against a Monte Carlo simulation:

In [4]:
def sorted_all_ge_mc(n):
    result = 0
    for i in range(n):
        a_scores = sorted(single_ability.sample() for i in range(6))
        b_scores = sorted(single_ability.sample() for i in range(6))
        if all(a >= b for a, b in zip(a_scores, b_scores)):
            result += 1
    return result / n

print(sorted_all_ge_mc(10000))

0.2075


In fact, we can also formulate the problem of "roll N dice, keep the M highest" in this way.
This is what underlies the `keep_highest()` method at the top.
While more efficient algorithms do exist for that specific problem, this is still fairly fast and reduces the amount of bespoke code. The evaluation simply sums all observed dice:

In [5]:
class SumPool(hdroller.EvalPool):
    """ A simple `EvalPool` that just sums the dice in a pool. """
    def next_state(self, state, outcome, count):
        """ Add the dice to the running total. """
        if state is None:
            return outcome * count
        else:
            return state + outcome * count
    
    def direction(self, *pools):
        """ This eval doesn't care about direction. """
        return 0

and the "keep highest" aspect is a property of the pool, which is already taken into account by the time `next_state()` sees the `count` parameter.

## Variant: at least one score strictly greater than

This is the chance that A's scores are strictly better than B's scores.

We can either subtract off the chance that A and B will have exactly the same sorted array, or we can explicitly encode it into the evaluation function:

In [9]:
class StrictlyBetter(hdroller.EvalPool):
    def next_state(self, state, outcome, a, b):
        # This time we explicitly store whether each side had some score up on the other.
        # This increases the state space and is therefore less efficient, but is still quite fast.
        advantage, a_had_one_up, b_had_one_up = state or (0, False, False)
        advantage += a - b
        if advantage > 0:
            a_had_one_up = True
        if advantage < 0:
            b_had_one_up = True
        return advantage, a_had_one_up, b_had_one_up
        
    def final_outcome(self, final_state, *_):
        _, a_had_one_up, b_had_one_up = final_state
        if a_had_one_up and not b_had_one_up:
            return 'a strictly better'
        elif b_had_one_up and not a_had_one_up:
            return 'b strictly better'
        elif not (a_had_one_up or b_had_one_up):
            return 'exactly the same'
        else:
            return 'mixed result'
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1
    
evaluator = StrictlyBetter()
print(evaluator.eval(single_ability.pool(6), single_ability.pool(6)))

Denominator: 22452257707354557240087211123792674816
|           Outcome |                                 Weight | Probability |
|------------------:|---------------------------------------:|------------:|
| a strictly better |  4696617436843743365666704597889926627 |  20.918241% |
| b strictly better |  4696617436843743365666704597889926627 |  20.918241% |
|  exactly the same |     5069150678806043464398031458429484 |   0.022577% |
|      mixed result | 13053953682988264465289403896554392078 |  58.140940% |

